# Лекция 7: Механизм внимания (attention)

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2021 г. 

При подготовке лекции использованы материалы:
* https://dlcourse.ai/
* https://neerc.ifmo.ru/wiki/index.php?title=Механизм_внимания

v 0.1 02.05.21

## Введение в механизм внимания



Изначально механизм внимания был представлен в контексте рекуррентных Seq2seq[1] сетей [2] для "обращения внимания" блоков декодеров на скрытые состояния RNN для любой итерации энкодера, а не только последней.

После успеха этой методики в __машинном переводе__ последовали ее внедрения в:
* других задачах обработки естественного языка
* задачах генерации описания изображения (в применении к CNN) 
* в генеративных состязательных сетях (GAN)

<img src="./img_new/seq2seq_translation.png" alt="" style="width: 500px;"/>

__Машинный перевод при помощи Encoder Decoder Network__:
* Имеем две RNN: энкодер и декодер. Обе модели принимают на вход токен и скрытое состояние, и выдают новый токен и скрытое состояние
* Энкодер последовательно принимает на вход все слова предложения на исходном языке
* Скрытое состояние на последнем слове является входным для декодера
* Для генерации перевода мы даем декодеру скрытое состояние энкодера на последнем токене и специальный стартовый токен, выход декодера опять даем ему на вход, и продолжаем этот процесс до получения токена, означающего конец предложения.

<center>         
    <img src="./img/seq_to_seq_v1.png" alt="" style="width: 200px;"/>
    <b>Рекуррентные сети Seq2seq (принципиальная схема)</b> <br/>
</center>

<center>         
    <img src="./img/enc_dec_2.png" alt="" style="width: 500px;"/>
    <b>Пример использования RNN для задачи машинного перевода</b> <br/>
</center>

__Специфика__:
* Для энкодера и декодера используются различные веса сети (RNN модуля)
* Скрытое состояние энкодера на последнем шаге обработки предложения (последовательности) является ключевым т.к. по сути кодирует все предложение. Затем декодер использует именно это состояние для того, чтобы сгенерировать перевод предложения на другом языке.
* Результат сэмплинга декодера на предыдущем шаге передается на следующий блок декодера в добавок к внутреннему состянию RNN декодера.

__Проблемы__:
* Расстояние между местом кодирования слова и местом его декодирования большое.
* Последовательность слов в исходном и целевом предложении часто должна быть разная.
* Часто количество слов в исходном и целевом предложении различается.


<br/>

__Приемы, направленные на преодоление проблемы__ :

* Для языков, в которых последовательность слов в эквивалентных передложениях примерно одинакова, может быть полезно декодеру __гененрировать предложение в обратном порядке__. Это дает возможность иметь небольшую последовательность преобразований (_"небольшое расстояние"_ ) от энкодинга слов (последних слов в исходном предложении) до места их декодирования.
* Может быть полезно скрытое состояние энкодера на последнем шаге передавать на вход каждому блоку декодера, чтобы каждый декодер имел прямой доступ к энкодингу всего предложения (этот подход тоже сокращает сложность передачи информации об исходном предложение "сквозь" последовательность в декодере):

<center>         
    <img src="./img/enc_dec_3.png" alt="" style="width: 500px;"/>
    <b>RNN в машинном переводе: использование ключевого состояния энкодера</b> <br/>
</center>


__Базовая архитектура Seq2seq__

_Seq2seq_ состоит из двух рекуррентных нейронных сетей (RNN): __Энкодера__ и __Декодера__.

<center>         
    <img src="./img/enc_dec_1.png" alt="" style="width: 400px;"/>
    <b>Рекуррентные сети Seq2seq</b> <br/>
</center>

* __Энкодер__ - принимает последовательность (например: предложение на языке _A_) и сжимает его в вектор скрытого состояния.
    * $x_i$ слова в предложении на языке _A_
    * $h_i$ скрытое состояние энкодера
    * __блоки энкодера__ (зеленый): получают на вход $x_i$ и передают скрытое состояние $h_i$ на следующую итерацию    
* __Декодер__ - выдает слово на языке _B_, принимает последнее скрытое состояние энкодера (для первого слова) / предыдущее скрытое состояние декодера (последующие слова) и предыдущее предсказанное слово.
    * $d_i$ скрытое состояние декодера
    * $y_i$ слова в предложении на языке _B_
    * Блоки декодера (фиолетовый) блоки декодера получающие на вход $y_{i-1}$ или специальный токен `<start>` (в случае первой итерации) и возвращаюшие на следующую итерацию:
        * $y_i$ - слова в предложении на языке _B_ 
        * $d_i$ - скрытое состояние декодера 
        * Перевод считается завершенным при возвращении $y_i$, равного специальному токену `<end>`

__Применение механизма внимания для Seq2seq: базовый принцип__

* Использование механизма внимания позволяет решать задачу нахождения закономерности между словами находящимися __на большом расстоянии друг от друга__. 
* LSTM, GRU и аналогичные блоки используются для улучшения передачи информации на большое количество итераций по сравнению с базовыми RNN, несмотря на это сохраняется проблема: влияние предыдущих состояний на текущее уменьшается экспоненциально от расстояния между словами
    * В классическом применении RNN результатом является только последнее скрытое состояние $h_m$, где $m$ - длина последовательности входных данных.
* __Механизм внимания__ улучшает этот показатель до __линейного__.
    * Использование механизма внимания позволяет использовать информацию, полученную не только из последнего скрытого состояния, но и из любого скрытого состояния $h_t$ для любого $t \in 1 \dotso m$.

__Устройство слоя механизма внимания__

<center>         
    <img src="./img/att_4.png" alt="" style="width: 400px;"/>
    <b>Обобщенный механизм внимания в RNN</b> <br/>
</center>  

1. Слой механизма внимания представляет собой обычную, чаще всего однослойную, нейронную сеть на вход которой подаются $h_t, t = 1 \ldots m$ (скрытые состояния на последовательных шагах RNN), а также вектор $d$ в котором содержится некий контекст, зависящий от конкретной задачи.
    * В случае _Seq2seq_ сетей вектором $d$ будет скрытое состояние предыдущей итерации декодера $d_{i-1}$ .

2. Выходом данного слоя будет является вектор $s$ (от _score_ ) - оценки на основании которых на скрытое состояние $h_i$ будет "обращено внимание".
3. Для нормализации значений $s$ используется $softmax$: $e = softmax(s)$
    * $\forall s\colon\  \sum_{i=1}^n softmax(e)_i = 1$
    * $\forall s,\ i\colon \ softmax(e)_i >= 0 $

4. Результатом работы слоя внимания является $c$ (__context vector__) который, содержит в себе информацию обо всех скрытых состояниях $h_i$ пропорционально нормализованному вниманию $e_i$ :

    * $с = \sum_{i=1}^m e_i h_i$
    
  

__Применение механизма внимания к базовой Seq2seq архитектуре__

При добавлении слоя механизма внимания в базовую архитектуру _Seq2seq_ между RNN _Энкодером_ и _Декодером_ получится следующая схема:

<center>         
    <img src="./img/att_5.png" alt="" style="width: 400px;"/>
    <b>Пример схемы работы Seq2seq сети с механизмом внимания</b> <br/>
</center>  

* $x_i, h_i, d_i, y_i$ имеют те же назначения, что и в базовом варианте без механизма внимания.
* _Агрегатор скрытых состояний энкодера (желтый)_ - агрегирует всю последовательность векторов $h = [h_1, h_2, h_3, h_4]$.
* _Блоки механизма внимания (красный)_ - принимают $h$ и $d_{i - 1}$ (_контекст для выбора параметров_ для блока $i$-го блока декодера (сиреневый цвет) ), возвращает $c_i$ - контекст для $i$-го блока декодера
* _Блоки декодера (фиолетовый)_ - по сравнению с обычной _Seq2seq_ сетью меняются входные данные. Теперь, вместо $y_{i-1}$  на $i$ итерации на вход подается: конкатенация $y_{i-1}$ и $c_i$ .

При помощи механизма внимания достигается __"фокусирование" декодера__ на определенных скрытых состояниях энкодера, на любом из шагов энкодера. 
* В случаях машинного перевода эта возможность помогает декодеру предсказывать на какие скрытые состояния при исходных определенных словах на языке _A_ необходимо обратить больше внимания при генерации очередного слова перевода на язык _B_.
    * При этом расстояние (в шагах последовательности) между генерируемым словом и влияющими на него словами из исходной последовательности никак не влияют на возможность использование скрытых состояний, сформированных под их воздействием.

<center>         
    <img src="./img/att_6.png" alt="" style="width: 300px;"/>
    <b>Пример весов механизма внимания при решении задачи машинного перевода</b> <br/>
</center>  


<br/>

<center>         
    <img src="./img/att_7.png" alt="" style="width: 450px;"/>
    <b>Эффект использования механизма внимания при решении задачи машинного перевода для предложений разной длины</b> <br/>
</center>  

__Использование механизмов внимания для других задач__

Задача _Image Captioning_ : на вход подается изображение, а на выходе создается текстовое предложение, описывающее визуальное содержание картинки.
* Классическая нейростевая модель для решения задачи Image Captioning состоит из:
    * сверточной нейронной сети (CNN) для извлечения визуальных характеристик из картинки
    * и рекуррентной нейронной сети (RNN) для перевода результата работы сверточной сети в текст.
    
* __Сверточный модуль внимания__ (сonvolutional block attention module) — модуль внимания для сверточных нейросетей. Применяется для задач детектирования обьектов на изображениях и классификации с входными данными больших размерностей. 

<center>         
    <img src="./img/att_8.png" alt="" style="width: 450px;"/>
    <b>Логика работы архитектуры, использующей attention model для задачи Image Captioning</b> <br/>
</center>

<br/><br/>

<center>         
    <img src="./img/att_9.png" alt="" style="width: 800px;"/>
    <b>Пример использования attention model для генерации существительных в описании картинок</b> <br/>
</center>

## Transformer

__Архитектура Transformer__

* Рассматриваем задачу машинного перевода _Seq2seq_
* новая архитектура для решения этой задачи, которая не является ни RNN, ни CNN

<center>         
    <img src="./img/att_12.png" alt="" style="width: 250px;"/>
    <b>Принципиальная архитектуры модели Transformer из статьи "Attention Is All You Need"*</b> <br/>
</center>

\*https://arxiv.org/abs/1706.03762

__Encoder, Multi-head attention__

__Multi-head attention__ - слой, который дает возможность каждому входному вектору (эмбеддингу слова) взаимодействовать с любыми другими словами предложения через attention mechanism, вместо:
* последовательной передачи hidden state, как в RNN
* свертки эмбеддингов соседних слов, как в CNN
    
<center>         
    <img src="./img/text_2.png" alt="" style="width: 450px;"/>
    <b>Пример иерархическая структуры предложения </b> <br/>
</center>

<center>         
    <img src="./img/att_11.png" alt="" style="width: 250px;"/>
    <b>Организация Multi-head attention</b> <br/>
</center>    

В Multi-head attention:
1. на вход даются вектора Query (Q), и несколько пар Key (K) и Value (V) (на практике, Key и Value это всегда один и тот же вектор)
2. каждый из них преобразуется обучаемым линейным преобразованием
3. вычисляется скалярное произведение Q со всеми K по очереди
4. результат этих скалярных произведений прходит через softmax
5. с полученными весами все вектора V суммируются в единый вектор
6. __Ключевое отличие от классического attention__: результат всех этих параллельных attention'ов (на картинке их $h$) конкатенируется, прогоняется через обучаемое линейное преобразование и идет на выход (получается вектор того же размера, что и каждый из входов)

Особенности:

* Q: в чем смысл множества одинаковых attention слоев?
    * Логика стандартного attention: сеть пытается выдать соответствие одного слова другому в предложении, если они близки чем-то
    * Не обучатся ли разныеслои  Multi-head attention в точности одному и тому же?
        * Нет. Технически, разных начальных случайных весов хватает, чтобы толкать разные слои в разные стороны.
        * Разные слои позволяют сети обращать внимание на различные аспекты слов (например семантические (смысловые) и грамматические аспекты слов)

* Так как на выход такой блок выдает вектор того же размера, что и был на входе, то этот блок можно вставлять в сеть несколько раз, добавляя сети глубину (в рассмотренной работе этот блок использовался с глубиной 6).
* Одной из фич каждого слова является __positional encoding__ — т.е. его позиция в предложении. Например, в процессе обработки слова это позволяет легко "обращать внимание" на соседние слова, если они важны.

<center>         
    <img src="./img/att__.gif" alt="" style="width: 450px;"/>
    <b>Пример работы архитектуры Transformer </b> <br/>
</center>

__Декодер__

<center>         
    <img src="./img/att_13.png" alt="" style="width: 400px;"/>
    <b>Организация декодера</b> <br/>
</center>

* декодер запускается по слову за раз, получает на вход:
    * вектора всех исходных слов, полученные при энкодинге
    * все прошлые слова и должен выдать следующее (на первой итерации получает специальный токен <start>).
* соответственно, декодер состоит из двух блоков:
    * первый — работает с векторами предыдущих декодированных слов, аналогичный использованному в процессе encoding (но может обращаться не ко всем слвам, а только к уже декодированным).
    * второй — работает с выходом энкодера. B этом случае Query — это вектор входа в декодере, а пары Key/Value — это финальные эмбеддинги энкодера
* все это снова повторяется 6 раз (глубокое обучение!), где выход предыдущего блока идет на вход следующему
* в конце сети стоит softmax, который выдает вероятности слов. Сэмплирование из него и есть результат, то есть следующее слово в предложении.
* Результат семплирования подается на вход следующему запуску декодера и процесс повторяется, пока декодер не выдаст токен `<EOS>`.

__Преимущества архитектуры Transformer__

(над RNN и CNN)

* Все слова в предложении одинаково доступны при кодировании/декодировании (расстояние не важно)
* Может эффективно выполняться параллельно
* Эффективная глубина сети меньше по сравнению с RNN (RNN "разматывается" вдоль последовательности), в Transformer глубина графа не зависит от длины последовательности

## Bert

__BERT__

__Bidirectional Encoder Representations from Transformers (BERT)__ - языковая модель, основанная на архитектуре трансформер, предназначенная для предобучения языковых представлений с целью их последующего применения в широком спектре задач обработки естественного языка. BERT был создан и опубликован в 2018 году Якобом Девлином и его коллегами из Google.
* BERT является автокодировщиком
* BERT использует трансформер-архитектуру
    * В каждом слое кодировщика применяется двустороннее внимание, что позволяет учитывать контекст с обеих сторон от рассматриваемого токена
* В каждом слое кодировщика применяется двустороннее внимание

<br/>

__Языковое моделирование__ (language modelling, LM) - использование различных статистических и вероятностных методов для определения вероятности того, что данная последовательность слов встречается в предложении.
* Для обеспечения достоверности предсказаний языковые модели строятся на основе анализа массивов текстовых данных. 
* Языковые модели используются в приложениях обработки естественного языка (natural language processing, NLP). В частности:
   * Задачах машинного перевода.
   * Задачах ответа на вопросы (question answering).
   * Других задачах, особенно, требующих генерацию текста.

<br/>
       
__Автокодировщик__ (autoencoder) — специальная архитектура искусственных нейронных сетей, позволяющая применять __обучение без учителя__ с использованием метода обратного распространения ошибки. 

Автокодировщик состоит из двух частей:
* __энкодера__ $g$, он переводит входной сигнал в его представление (код): $h = g(x)$
* __декодера__ $f$, он восстанавливает сигнал по его коду: $x = f(h)$

При этом семейства функций энкодера $g$ и декодера $f$ определенным образом ограничены, чтобы автоэнкодер был вынужден отбирать наиболее важные свойства сигнала.

Автокодировщик, изменяя (обучая) $f$ и $g$, стремится выучить тождественную функцию $x = f(g(x))$, минимизируя заданный функционал ошибки: $L(x, f(g(x)))$.

Автокодировщик можно использовать:
* для предобучения, например, когда стоит задача классификации, а размеченных пар слишком мало
* понижения размерности в данных, например для последующей визуализации
* когда надо научиться различать полезные свойства входного сигнала       

__Обучение модели BERT__

BERT-модель __предварительно обучена без учителя__ на на двух NLP-задачах:
* генерации пропущенного токена (masked language modeling)
    * На вход BERT подаются токенизированные пары предложений, в которых __некоторые токены скрыты__
    * Модель обучается генерировать пропущенные токены (слова или отдельные токены)

<center>         
    <img src="./img/att_14.png" alt="" style="width: 600px;"/>
    <b>Логика задачи генерации пропущенного токена</b> <br/>
</center>

* предсказание следующего предложения:
    * Задача же предсказания следующего предложения есть задача бинарной классификации — является ли второе предложение продолжением первого
    * Благодаря этой задаче сеть обучается различать связь между предложениями в тексте

<center>         
    <img src="./img/att_15.png" alt="" style="width: 600px;"/>
    <b>Логика задачи генерации пропущенного токена</b> <br/>
</center>



<center>         
    <img src="./img/att_16.png" alt="" style="width: 600px;"/>
    <b>Представление входных данных модели</b> <br/>
</center>

* Токенизация в BERT: 
    * Токенами служат слова, доступные в словаре, или их составные части
    * Если слово отсутствует в словаре, оно разбивается на части, которые в словаре присутствуют
    * Словарь является составляющей модели
        * Например в BERT-Base используется словарь примерно из 30K слов. 
* В нейронной сети BERT __токены кодируются своими эмбеддингами__, а именно: соединяются представления самого токена (предобученные), номера его предложения, а также позиции токена внутри своего предложения.

* Предобучение проведено на больших корпусах текстов (есть многоязычные варианты модели BERT).
    * В частности, многоязыковая модель BERT-Base, поддерживающая 104 языка, состоит из 12 слоев, 768 узлов, 12 выходов и 110M параметров
    
* Предобученные модели можно подстраивать под решение конкретных NLP-задач:
    * Дообучаем модель на данных, специфичных задаче: знание языка уже получено на этапе предобучения языковой модели, необходима лишь коррекция сети

__Данные и оценка качества__

Исходные данные: Предобучение ведётся на текстовых данных корпуса BooksCorpus (800 млн. слов), а также на текстах англоязычной Википедии (2.5 млрд. слов). 


Качество модели авторы оценивают на популярном для обучения моделей обработки естественного языка наборе задач GLUE ( https://gluebenchmark.com/tasks )

__Гиперпараметры модели__

Гиперпараметрами модели являются:
* $H$ — размерность скрытого пространства кодировщика
* $L$ — количество слоёв-кодировщиков
* $A$ — количество голов в механизме внимания.


В репозитории Google Research доступны для загрузки и использования несколько вариантов обученной сети в формате контрольных точек обучения модели популярного фреймворка TensorFlow[5]. В таблице в репозитории приведено соответствие параметров L и H и моделей. Использование моделей с малыми значениями гиперпараметров на устройствах с меньшей вычислительной мощностью позволяет сохранять баланс между производительностью и потреблением ресурсов. Также представлены модели с различным типом скрытия токенов при обучении, доступны два варианта: скрытие слова целиком (англ. whole word masking) или скрытие составных частей слов (англ. WordPiece masking).

Также модель доступна для использования с помощью популярной библиотеки PyTorch.

__Точная настройка (Fine-tuning)__

Этот этап обучения зависит от задачи, и выход сети, полученной на этапе предобучения, может использоваться как вход для решаемой задачи. 

_Пример:_ если решаем задачу построения вопросно-ответной системы, можем использовать в качестве ответа последовательность токенов, следующую за разделителем предложений. В общем случае дообучаем модель на данных, специфичных задаче: знание языка уже получено на этапе предобучения, необходима лишь коррекция сети.

Интерпретация этапа fine-tuning — обучение решению конкретной задачи при уже имеющейся общей модели языка.

__Пример использования__

Пример предказания пропущенного токена при помощи BERT в составе PyTorch. Скрытый токен — первое слово второго предложения:

In [ ]:
# Загрузка токенизатора и входные данные
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"

# Токенизация ввода, также добавляются специальные токены начала и конца предложения.
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Конвертирвание ввода в формат тензоров PyTorch
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])
encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)

# Выбираем токен, который будет скрыт и позднее предсказан моделью
masked_index = 8
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])

# Загрузка модели
masked_lm_model = torch.hub.load('huggingface/pytorch-transformers', 'modelWithLMHead', 'bert-base-cased')
predictions = masked_lm_model(tokens_tensor, token_type_ids=segments_tensors)

# Предсказание скрытого токена
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'Jim'